Portfolio 3

Hej Henriette
Her er hvad jeg har nået - det bærer meget præg af at være skitse, samt at jeg har haft nogle forhindringer der tog mig lidt tid at få bugt med. Men jeg ville alligevel aflevere inden deadline, så jeg kunne få feedback

Mvh Frederik

1) Shålå - her besvarer vi med info fra https://www.smk.dk/article/smk-api/ - det er fint

2) Sammensæt en URL der søger efter kulturinstitutionens elementer (fx. genstande, kunstværker, personer, avisartikler, osv.) i APIen (gerne 100+ stk) med output i JSON. Load denne URL i en JSON beautifier og beskriv indholdet:

Jeg har ikke været på SMK siden 2012, men jeg syntes godt om den daværende Matisse-udstilling. Derfor har jeg valgt at søge på Matisse. 

Min søgning på SMK's 'Swagger' var under art/search, og mine søgeord: Matisse, Henri. Herudover har jeg i query field markeret "Creator".

Det resulterende URL https://collection.smk.dk/#q=henri%20matisse&fq=artist_surname_firstname%3A%22Matisse%20Henri%22 Har jeg sat ind i beautifieren på https://codebeautify.org/jsonviewer 

Heraf fremgår det, at der i SMK's database er 199 værker skabt af Matisse. Når jeg krydsreferer med søgning på https://collection.smk.dk og direkte i feltet "kunstner/producent" søger på Henri Matisse, får jeg 197 hits. HVORFOR ER DER TO MERE I DEN ENE?

a) Hvilken typer metadata er der for hver element?
Når jeg kører mit URL i beautifieren får jeg 10 eksempler ud af de 199 resultater. Hver har mellem 24-41 kategorier af metadata. (LISTE??). Herunder detaljeret information om anskaffelse og katalogisering af værkerne. Endvidere har alle værkerne opgørelse over teknikker/techniques (fx Kul på papir), produktionsdato/production_date, værkets størrelse (bl.a. dimensions), beskrivelse af motivet, 'colors', der har hex-farvekoderne for hvert værks faver og meget mere. Dvs Størstedelen er deskriptive data om det enkelte værk. Herudover er der informationer som "on_display", "public_domain", "has_image" der beskriver museets FORHLD TIL BILLEDET

b)Sammenlign metadata med Dublin Core. Er der overensstemmelse?
SE SLIDES for heritage_101.2 - Men ja, det passer ret godt - bare match det op, ord for ord

c)Er der nogle tal der kan bruges til statistiske beregninger (fx. årstal)?
Ja! Med acquisition_date_precision og/eller production_date ville man f.eks. kunne beregne statistik på hvornår værkerne er skabt og/eller hvornår museet har fået fat i dem. - mest interessant ville måske være, at se på hvornår Matisse var mest produktiv.
Herudover kunne man f.eks. lave statistik over farvebrug med hex-farvekoderne, og f.eks. finde ud af om enkelte præcise farver er overrepræsenteret - havde Henri Matisse en yndlingsfarve?
Man kan også lave statistik på, hvilken teknik der er met fremkommende hos Matisse i SMK-samlingen (techniques)

d)EKSEMPLER HVORDAN???


3)

In [651]:
# importer requests modulet
import requests
# importer pandas
import pandas as pd
# importer numpy
import numpy as np
# importer json normalize modulet
from pandas.io.json import json_normalize

In [652]:
# en variabel for mit API URL - (v1)
api_search_url = 'https://api.smk.dk/api/v1/art/search/'

params = {
    'keys': 'Matisse', 
    'rows': 200,
    'offset':0
}

response = requests.get(api_search_url, params=params)
print(response)

<Response [200]>


In [653]:
# retrieve the JSON from the response variable and add to the json variable
json = response.json()

# flatten the JSON file object "article" into a dataframe called df
df = json_normalize(json['items']) 
# check the first 5 rows
df

,id,created,modified,acquisition_date_precision,responsible_department,distinguishing_features,frame_notes,dimensions,documentation,inscriptions,...,image_native,image_cropped,image_orientation,colors,labels,related_objects,content_description,collection,part_of,parts
0,1180043169_object,2019-08-07T05:39:26Z,2019-08-08T07:50:25Z,1976-12-30,Samling og Forskning (KKS),[Gulnet papir],"[Bagklædning: false, Mikroklimaramme: false]","[{'part': 'bladmaal', 'type': 'hojde', 'unit':...",[{'notes': 'p. 13'}],[{'language': 'eng'}],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1180076925_object,2019-08-07T07:51:32Z,2019-08-12T09:49:10Z,1928-12-31,Samling og Forskning (KMS),NaN,"[Bagklædning: true, Mikroklimaramme: true]","[{'notes': '550 x 460 mm', 'part': 'netto', 't...","[{'title': 'Henri Matisse', 'author': 'Pierre ...","[{'content': 'Henri-Matisse', 'position': 'f.n...",...,https://iip.smk.dk/iiif/jp2/KMSr78.TIF.reconst...,True,portrait,"[#D2D2D2, #555555, #9B9B9B, #5A5A5A, #9B9B9B]",NaN,NaN,NaN,NaN,NaN,NaN
2,1180064772_object,2019-08-07T07:02:38Z,2019-08-12T09:44:42Z,1936-12-31,Samling og Forskning (KMS),NaN,"[Bagklædning: false, Mikroklimaramme: false]","[{'notes': '405 x 325 mm', 'part': 'netto', 't...",[{'title': 'Tilskueren: maanedsskrift=månedssk...,"[{'content': 'Henri-Matisse ', 'position': 'f....",...,https://api.smk.dk/api/v1/thumbnail/080a0800-e...,NaN,na,"[#863B7A, #F4C6A3, #555555, #9B9B9B, #037676]",[{'text': 'Den danske forbindelse Henri Matiss...,NaN,NaN,NaN,NaN,NaN
3,1180078562_object,2019-08-07T07:58:20Z,2019-08-12T09:49:47Z,1928-12-31,Samling og Forskning (KMS),NaN,"[Bagklædning: false, Mikroklimaramme: true]","[{'notes': '820 x 935 mm', 'part': 'netto', 't...",[{'title': 'Chr. Tetzen-Lunds Samling af moder...,"[{'content': 'Henri-Matisse ', 'position': 'f....",...,https://iip.smk.dk/iiif/jp2/KMSr82_-_cropped.t...,NaN,landscape,"[#555555, #555555, #000000, #9B9B9B, #5A5A5A]",NaN,NaN,NaN,NaN,NaN,NaN
4,1180044362_object,2019-08-07T05:43:12Z,2019-08-12T09:36:35Z,1928-12-31,Samling og Forskning (KMS),NaN,"[Bagklædning: false, Mikroklimaramme: false]","[{'notes': '2198 x 1474 x 68 mm', 'part': 'bru...","[{'title': 'Histoire de l'art contemporain', '...","[{'content': 'Henri-Matisse ', 'position': 'f....",...,https://iip.smk.dk/iiif/jp2/KMSKMSr76_fra_glas...,NaN,portrait,"[#9B9B9B, #5A5A5A, #5A5A5A, #9B9B9B, #555555]",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1180077680_object,2019-08-07T07:54:36Z,2019-08-12T09:49:26Z,1941-12-31,Samling og Forskning (KKS),NaN,"[Bagklædning: false, Mikroklimaramme: false]","[{'notes': '330 x 250 mm', 'part': 'bladmaal',...",[{'title': 'Henri Matisse: catalogue raisonné ...,[{'content': 'Påskrift med fremmed hånd f.n.t....,...,https://iip.smk.dk/iiif/jp2/KKS13429-32.TIF.re...,True,portrait,"[#FFFFFF, #FFFFFF, #FFFFFF, #FFFFFF, #D2D2D2]",NaN,"[{'notes': 'Ekstra tryk af blad 3 til ""Stéphan...",NaN,NaN,[KKS13429],NaN
196,1180077736_object,2019-08-07T07:54:48Z,2019-08-12T09:49:28Z,1941-12-31,Samling og Forskning (KKS),NaN,"[Bagklædning: false, Mikroklimaramme: false]","[{'notes': '330 x 250 mm', 'part': 'bladmaal',...",[{'title': 'Henri Matisse: catalogue raisonné ...,[{'content': 'Påskrift med fremmed hånd f.n.t....,...,https://iip.smk.dk/iiif/jp2/KKS13429-24.TIF.re...,True,portrait,"[#FFFFFF, #FFFFFF, #FFFFFF, #D2D2D2, #D2D2D2]",NaN,"[{'notes': 'Ekstra tryk af blad 24 til ""Stépha...",NaN,NaN,[KKS13429],NaN
197,1180078107_object,2019-08-07T07:56:25Z,2019-08-12T09:49:37Z,1941-12-31,Samling og Forskning (KKS),NaN,"[Bagklædning: false, Mikroklimaramme: false]","[{'notes': '250 x 330 mm', 'part': 'bladmaal',...",[{'title': 'Henri Matisse: catalogue raisonné ...,[{'content': 'Påskrift med fremmed hånd f.n.t....,...,https://iip.smk.dk/iiif/jp2/KKS13429-46.TIF.re...,True,portrait,"[#FFFFFF, #FFFFFF, #FFFFFF, #FFFFFF, #D2D2D2]",NaN,"[{'notes': 'Ekstra tryk af blad 17 ftil ""Stéph...",NaN,NaN,[KKS13429],NaN
198,1180082384_objec

I det ovenstående har jeg ændret på mine parametre, da resultatet var et "empty dataframe". Jeg fjernede parametret 'qfields', der var "creator", hvorefter det lykkedes - dog kun med en dataframe bestående af 7 rækker og 44 kolonner. Efterfølgende ændrede jeg mit søgeord til kun at være 'Matisse', og satte til 200 rows.
Når jeg søger på 'Matisse' på https://collection.smk.dk/#/en/q=Matisse får jeg kun 197 resultater, men når jeg beautifier min søgning fra 'Swagger', får jeg 268 resultater. Jeg har valgt at gå videre alligvel (men begrænset mit sæt til 200.

In [654]:
df.shape
# Jeg brugte shape til at kontrollere at min resulterende dataframe gav mening, da jeg som ovenstående beskrevet flere gange endte med 7,44

(200, 48)

Ovenstående resultat giver mig en dataframe med 200 rækker og 48 kolonner. De 48 kolonner er de forskellige typer metadata for hvert objekt.


3) a i: Hvilke data og datatyper er der? 

In [655]:
df.dtypes 
# Jeg laver en oversigt over kolonnernes forskellige datatyper

id                             object
created                        object
modified                       object
acquisition_date_precision     object
responsible_department         object
distinguishing_features        object
frame_notes                    object
dimensions                     object
documentation                  object
inscriptions                   object
object_names                   object
production                     object
production_date                object
techniques                     object
titles                         object
notes                          object
object_history_note            object
object_number                  object
iiif_manifest                  object
work_status                    object
public_domain                    bool
rights                         object
on_display                       bool
has_image                        bool
credit_line                    object
current_location_name          object
exhibitions 

3) a, ii Hvor mange kolonner og rækker

In [656]:
df.shape
# Stadig som det skal være

(200, 48)

Der er en længere række af kolonner, jeg ikke er interesseret i, så dem fjerner vi

In [657]:
to_drop = ['created',
           'modified',
           'responsible_department',
           'distinguishing_features',
           'frame_notes',
           'dimensions',
           'inscriptions',
           'object_names',
           'work_status',
           'public_domain',
           'rights',
           'has_image',
           'related_objects',
           'image_native',
           'image_cropped',
           'labels',
           'collection',
           'part_of',
           'number_of_parts',
           'object_number',
           'iiif_manifest',
           'exhibitions',
           'image_thumbnail',
          'image_iiif_id',
          'image_mime_type',
          'image_iiif_info',
          'documentation',
          'on_display',
          'notes',
          'credit_line',
          'current_location_name',
          'production_dates_notes',
          'alternative_images',
          'content_description',
          'parts']

# drop the columns listed in "to_drop" in the dataframe
df.drop(to_drop, inplace=True, axis=1)
# view the first 5 rows of the df
df.head()


,id,acquisition_date_precision,production,production_date,techniques,titles,object_history_note,materials,image_width,image_height,image_size,image_orientation,colors
0,1180043169_object,1976-12-30,"[{'creator': 'Giersing, Harald', 'creator_date...","[{'start': '1919-12-31T00:00:00.000Z', 'end': ...",[{'technique': 'Blyant'}],"[{'title': 'Henri Matisse', 'language': 'da-DK...","[Har tilhørt fru Besse Giersing, derefter tilh...",NaN,NaN,NaN,NaN,NaN,NaN
1,1180076925_object,1928-12-31,"[{'creator': 'Matisse, Henri', 'creator_date_o...","[{'start': '1906-01-01T00:00:00.000Z', 'end': ...",[{'technique': 'Olie på lærred'}],"[{'title': 'Selvportræt', 'language': 'da-DK',...",NaN,"[{'material': 'lærred'}, {'material': 'olie'}]",2232.0,2657.0,16424188.0,portrait,"[#D2D2D2, #555555, #9B9B9B, #5A5A5A, #9B9B9B]"
2,1180064772_object,1936-12-31,"[{'creator': 'Matisse, Henri', 'creator_date_o...","[{'start': '1905-01-01T00:00:00.000Z', 'end': ...",[{'technique': 'Olie på lærred'}],[{'title': 'Portræt af Madame Matisse. Den grø...,[March-april 1906 købt af Sarah og Michael Ste...,"[{'material': 'lærred'}, {'material': 'olie'}]",NaN,NaN,29613636.0,na,"[#863B7A, #F4C6A3, #555555, #9B9B9B, #037676]"
3,1180078562_object,1928-12-31,"[{'creator': 'Matisse, Henri', 'creator_date_o...","[{'start': '1912-01-01T00:00:00.000Z', 'end': ...",[{'technique': 'Olie på lærred'}],"[{'title': 'Guldfiskene', 'language': 'da-DK',...",NaN,"[{'material': 'lærred'}, {'material': 'olie'}]",2903.0,2542.0,22171528.0,landscape,"[#555555, #555555, #000000, #9B9B9B, #5A5A5A]"
4,1180044362_object,1928-12-31,"[{'creator': 'Matisse, Henri', 'creator_date_o...","[{'start': '1907-01-01T00:00:00.000Z', 'end': ...",[{'technique': 'Limfarve på lærred'}],"[{'title': 'Le Luxe II', 'language': 'da-DK', ...",NaN,"[{'material': 'lærred'}, {'material': 'limfarv...",3117.0,4614.0,43178280.0,portrait,"[#9B9B9B, #5A5A5A, #5A5A5A, #9B9B9B, #555555]"


Her opstår dog problemet at mange af vores metadata er 'pakket ned', og derfor svære at arbejde med. I det følgende, vil jeg derfor "pakke dem ud" (flatten), for bedre at kunne tilgå mine metadata.

In [658]:
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out



In [659]:
dic_flattened = [flatten_json(d) for d in json['items']]
df= json_normalize(dic_flattened)

In [660]:
df.shape
# Som vi kan se, er der nu 806 kolonner at gøre godt med. Så jeg kører lige endnu en oprydningsrunde.

(200, 806)

In [661]:
df.head()

,id,created,modified,acquisition_date_precision,responsible_department,distinguishing_features_0,frame_notes_0,frame_notes_1,dimensions_0_part,dimensions_0_type,...,alternative_images_6_orientation,alternative_images_7_mime_type,alternative_images_7_iiif_id,alternative_images_7_iiif_info,alternative_images_7_width,alternative_images_7_height,alternative_images_7_size,alternative_images_7_thumbnail,alternative_images_7_native,alternative_images_7_orientation
0,1180043169_object,2019-08-07T05:39:26Z,2019-08-08T07:50:25Z,1976-12-30,Samling og Forskning (KKS),Gulnet papir,Bagklædning: false,Mikroklimaramme: false,bladmaal,hojde,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1180076925_object,2019-08-07T07:51:32Z,2019-08-12T09:49:10Z,1928-12-31,Samling og Forskning (KMS),NaN,Bagklædning: true,Mikroklimaramme: true,netto,hojde,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1180064772_object,2019-08-07T07:02:38Z,2019-08-12T09:44:42Z,1936-12-31,Samling og Forskning (KMS),NaN,Bagklædning: false,Mikroklimaramme: false,netto,hojde,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1180078562_object,2019-08-07T07:58:20Z,2019-08-12T09:49:47Z,1928-12-31,Samling og Forskning (KMS),NaN,Bagklædning: false,Mikroklimaramme: true,netto,hojde,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1180044362_object,2019-08-07T05:43:12Z,2019-08-12T09:36:35Z,1928-12-31,Samling og Forskning (KMS),NaN,Bagklædning: false,Mikroklimaramme: false,brutto,hojde,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [662]:
to_drop = ['created',
           'modified',
           'responsible_department',
           'credit_line_0',
          'distinguishing_features_0',
          'distinguishing_features_1',
          'frame_notes_0',
          'frame_notes_1',
          'dimensions_0_part',
          'dimensions_1_part',
          'dimensions_2_part',
          'dimensions_3_part',
          'dimensions_4_part',
          'dimensions_5_part',
          'dimensions_0_type',
          'dimensions_1_type',
          'dimensions_2_type',
          'dimensions_3_type',
          'dimensions_4_type',
          'dimensions_5_type',
          'dimensions_0_unit',
          'dimensions_1_unit',
          'dimensions_2_unit',
          'dimensions_3_unit',
          'dimensions_4_unit',
          'dimensions_5_unit',
          'dimensions_0_value',
          'dimensions_1_value',
          'dimensions_2_value',
          'dimensions_3_value',
          'dimensions_4_value',
          'dimensions_5_value',
          'dimensions_0_notes',
          'dimensions_1_notes',
          'dimensions_2_notes',
          'dimensions_3_notes',
          'dimensions_4_notes',
          'dimensions_5_notes',
          'alternative_images_0_mime_type',
          'alternative_images_1_mime_type',
          'alternative_images_2_mime_type',
          'alternative_images_3_mime_type',
          'alternative_images_4_mime_type',
          'alternative_images_5_mime_type',
          'alternative_images_6_mime_type',
          'alternative_images_7_mime_type',
          'alternative_images_0_iiif_id',
          'alternative_images_1_iiif_id',
          'alternative_images_2_iiif_id',
          'alternative_images_3_iiif_id',
          'alternative_images_4_iiif_id',
          'alternative_images_5_iiif_id',
          'alternative_images_6_iiif_id',
          'alternative_images_7_iiif_id',
          'alternative_images_0_iiif_info',
          'alternative_images_1_iiif_info',
          'alternative_images_2_iiif_info',
          'alternative_images_3_iiif_info',
          'alternative_images_4_iiif_info',
          'alternative_images_5_iiif_info',
          'alternative_images_6_iiif_info',
          'alternative_images_7_iiif_info',
          'alternative_images_0_width',
          'alternative_images_1_width',
          'alternative_images_2_width',
          'alternative_images_3_width',
          'alternative_images_4_width',
          'alternative_images_5_width',
          'alternative_images_6_width',
          'alternative_images_7_width',
          'alternative_images_0_height',
          'alternative_images_1_height',
          'alternative_images_2_height',
          'alternative_images_3_height',
          'alternative_images_4_height',
          'alternative_images_5_height',
          'alternative_images_6_height',
          'alternative_images_7_height',
          'alternative_images_0_size',
          'alternative_images_1_size',
          'alternative_images_2_size',
          'alternative_images_3_size',
          'alternative_images_4_size',
          'alternative_images_5_size',
          'alternative_images_6_size',
          'alternative_images_7_size',
          'alternative_images_0_thumbnail',
          'alternative_images_1_thumbnail',
          'alternative_images_2_thumbnail',
          'alternative_images_3_thumbnail',
          'alternative_images_4_thumbnail',
          'alternative_images_5_thumbnail',
          'alternative_images_6_thumbnail',
          'alternative_images_7_thumbnail',
          'alternative_images_0_native',
          'alternative_images_1_native',
          'alternative_images_2_native',
          'alternative_images_3_native',
          'alternative_images_4_native',
          'alternative_images_5_native',
          'alternative_images_6_native',
          'alternative_images_7_native',
          'alternative_images_0_orientation',
          'alternative_images_1_orientation',
          'alternative_images_2_orientation',
          'alternative_images_3_orientation',
          'alternative_images_4_orientation',
          'alternative_images_5_orientation',
          'alternative_images_6_orientation',
          'alternative_images_7_orientation',
          'image_mime_type',
          'image_iiif_id',
          'image_iiif_info',
          'image_width',
          'image_height',
          'image_size',
          'image_thumbnail',
          'image_native',
          'image_cropped',
          'image_orientation']
# drop the columns listed in "to_drop" in the dataframe
df.drop(to_drop, inplace=True, axis=1)
# view the first 5 rows of the df
df.shape

(200, 686)

Hvad jeg erfarede efter mit flatten-stunt, var at nu var alle de kolonner jeg havde fjernet fra mit dataframe, genopstået - sikkert fordi jeg jo har lavet et nyt dataframe

Så - hvad der er sket her er, at jeg efter at have copy-pastet 'image_documentation_0_title' osv op til omkring 90, besluttede jeg mig for, at der måtte være en smartere måde. så i de neenstående har jeg forsøgt med filter-funktion og regex at fjerne 'documentation'-kolonnerne. Først kun 'author' og efterfølgende alle documentation
Det min regex gør her er, at finde alle kolonner der hedder noget med 'documentation_' efterfulgt af ml. 1 og 3 decimaler og endnu et underscore efterfulgt af hvilket som helst ord

In [663]:
df = df[df.columns.drop(list(df.filter(regex='documentation_[0-9]{1,3}_author')))]

In [664]:
df = df[df.columns.drop(list(df.filter(regex='documentation_[0-9]{1,3}_([^\s]+)')))]

In [665]:
df.shape

(200, 226)

In [666]:
df.head()

,id,acquisition_date_precision,inscriptions_0_language,object_names_0_name,production_0_creator,production_0_creator_date_of_birth,production_0_creator_date_of_death,production_0_creator_nationality,production_0_creator_history,production_0_notes,...,parts_74,parts_75,parts_76,content_description_1,inscriptions_0_notes,titles_0_notes,titles_1_notes,inscriptions_2_content,inscriptions_3_content,inscriptions_4_content
0,1180043169_object,1976-12-30,eng,tegning,"Giersing, Harald",1881-04-24T00:00:00.000Z,1927-01-15T00:00:00.000Z,dansk,"Tegninger: Td 795, 796, 797, 798, 799, Skitseb...",Efter Matisse' Selvportræt (1906) i Rumps samling,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1180076925_object,1928-12-31,NaN,maleri,"Matisse, Henri",1869-01-01T00:00:00.000Z,1954-01-01T00:00:00.000Z,fransk,"Tegninger: Tu 33a, Tu 380c + pennetegning i bo...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1180064772_object,1936-12-31,NaN,maleri,"Matisse, Henri",1869-01-01T00:00:00.000Z,1954-01-01T00:00:00.000Z,fransk,"Tegninger: Tu 33a, Tu 380c + pennetegning i bo...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1180078562_object,1928-12-31,NaN,maleri,"Matisse, Henri",1869-01-01T00:00:00.000Z,1954-01-01T00:00:00.000Z,fransk,"Tegninger: Tu 33a, Tu 380c + pennetegning i bo...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1180044362_object,1928-12-31,NaN,maleri,"Matisse, Henri",1869-01-01T00:00:00.000Z,1954-01-01T00:00:00.000Z,fransk,"Tegninger: Tu 33a, Tu 380c + pennetegning i bo...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Jeg fortsætter lige oprydningen lidt endnu

In [667]:
df = df[df.columns.drop(list(df.filter(regex='exhibitions_[0-9]{1,3}_([^\s]+)')))]
# Samme regex som ovenståeden, jeg har blot byttet 'documentations_' ud med 'exhibitions_'

In [668]:
df.shape #og nedskaleringen fortsætter (jeg er nede på 163 kolonner nu)

(200, 163)

In [669]:
df.head()
# Jeg tjekker løbende mit df.head, for at få et overblik, hvorefter jeg finder de kolonner jeg ønsker at fjerne, og evt krydsreferer med min beautifier

,id,acquisition_date_precision,inscriptions_0_language,object_names_0_name,production_0_creator,production_0_creator_date_of_birth,production_0_creator_date_of_death,production_0_creator_nationality,production_0_creator_history,production_0_notes,...,parts_74,parts_75,parts_76,content_description_1,inscriptions_0_notes,titles_0_notes,titles_1_notes,inscriptions_2_content,inscriptions_3_content,inscriptions_4_content
0,1180043169_object,1976-12-30,eng,tegning,"Giersing, Harald",1881-04-24T00:00:00.000Z,1927-01-15T00:00:00.000Z,dansk,"Tegninger: Td 795, 796, 797, 798, 799, Skitseb...",Efter Matisse' Selvportræt (1906) i Rumps samling,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1180076925_object,1928-12-31,NaN,maleri,"Matisse, Henri",1869-01-01T00:00:00.000Z,1954-01-01T00:00:00.000Z,fransk,"Tegninger: Tu 33a, Tu 380c + pennetegning i bo...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1180064772_object,1936-12-31,NaN,maleri,"Matisse, Henri",1869-01-01T00:00:00.000Z,1954-01-01T00:00:00.000Z,fransk,"Tegninger: Tu 33a, Tu 380c + pennetegning i bo...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1180078562_object,1928-12-31,NaN,maleri,"Matisse, Henri",1869-01-01T00:00:00.000Z,1954-01-01T00:00:00.000Z,fransk,"Tegninger: Tu 33a, Tu 380c + pennetegning i bo...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1180044362_object,1928-12-31,NaN,maleri,"Matisse, Henri",1869-01-01T00:00:00.000Z,1954-01-01T00:00:00.000Z,fransk,"Tegninger: Tu 33a, Tu 380c + pennetegning i bo...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [670]:
df = df[df.columns.drop(list(df.filter(regex='parts_[0-9]{1,3}')))]
# Jeg fjerner alle kolonnerne der hedder noget med 'parts_' efterfulgt af 1-3 decimaler

In [671]:
df = df[df.columns.drop(list(df.filter(regex='inscriptions_[0-9]{1,3}_([^\s]+)')))]

df = df[df.columns.drop(list(df.filter(regex='titles_[0-9]{1,3}_([^\s]+)')))]

df = df[df.columns.drop(list(df.filter(regex='related_objects_[0-9]{1,3}_([^\s]+)')))]

df = df[df.columns.drop(list(df.filter(regex='part_of_[0-9]{1,3}')))]





# Samme metode som før, men nu fjerner jeg 'inscriptions', 'related_objects' 'titles', 'part_of'





In [672]:
df.shape

(200, 57)

In [673]:
df.dtypes
# siden jeg nu er nede på 57 kolonner kører jeg en 'dtypes', mest for at få et overblik over hvilke kolonner der stadig er tilbage

id                                     object
acquisition_date_precision             object
object_names_0_name                    object
production_0_creator                   object
production_0_creator_date_of_birth     object
production_0_creator_date_of_death     object
production_0_creator_nationality       object
production_0_creator_history           object
production_0_notes                     object
production_0_creator_lref              object
production_1_creator                   object
production_1_creator_date_of_birth     object
production_1_creator_date_of_death     object
production_1_creator_nationality       object
production_1_creator_history           object
production_1_creator_role              object
production_1_creator_lref              object
production_date_0_start                object
production_date_0_end                  object
production_date_0_period               object
techniques_0_technique                 object
notes_0                           

In [674]:
df = df[df.columns.drop(list(df.filter(regex='alternative_images_[0-9]{1,3}_([^\s]+)')))]

df = df[df.columns.drop(list(df.filter(regex='labels_[0-9]{1,3}_([^\s]+)')))]

# Ovenstående fjerner henholdsvis alternative_images og labels


In [675]:
df.shape
# Hvilket overhovedet ikke virkede. Men jeg tager udgangspunkt i min dtypes-liste og beautifier, og fjerner de uønskede kolonner via df.drop 

(200, 51)

In [676]:
to_drop = ['production_0_creator_date_of_birth',     
           'production_0_creator_date_of_death',     
           'production_0_creator_nationality',
           'production_0_creator_history',           
           'production_0_notes',                     
           'production_0_creator_lref',             
           'production_1_creator',
           'production_1_creator_date_of_birth',
           'production_1_creator_date_of_death',     
           'production_1_creator_nationality',
           'production_1_creator_history',           
           'production_1_creator_role',              
           'production_1_creator_lref',              
           'notes_0',                
           'object_history_note_0',
           'object_number',
           'iiif_manifest',
           'public_domain',
           'rights',
           'current_location_name',
           'number_of_parts',
           'production_dates_notes_0',
           'content_description_0',
           'collection_0',
           'production_0_place',
           'production_2_creator_role',
           'production_2_creator_lref',
           'content_description_1',
           'production_date_1_start',                
           'production_date_1_end',
           'production_date_0_start',                
           'production_date_0_end',
           'production_1_creator',
           'production_2_creator']
           
           
           
# drop the columns listed in "to_drop" in the dataframe
df.drop(to_drop, inplace=True, axis=1)           

In [677]:
df.shape 
# Og nu ser det (langt om længe) ud til at være en lidt mere overskuelig størrelse

(200, 18)

In [678]:
df.head()

,id,acquisition_date_precision,object_names_0_name,production_0_creator,production_date_0_period,techniques_0_technique,work_status_0,on_display,has_image,materials_0_material,materials_1_material,colors_0,colors_1,colors_2,colors_3,colors_4,techniques_1_technique,production_date_1_period
0,1180043169_object,1976-12-30,tegning,"Giersing, Harald",Ca. 1922,Blyant,kopi,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1180076925_object,1928-12-31,maleri,"Matisse, Henri",(1906),Olie på lærred,NaN,True,True,lærred,olie,#D2D2D2,#555555,#9B9B9B,#5A5A5A,#9B9B9B,NaN,NaN
2,1180064772_object,1936-12-31,maleri,"Matisse, Henri",(1905),Olie på lærred,NaN,True,True,lærred,olie,#863B7A,#F4C6A3,#555555,#9B9B9B,#037676,NaN,NaN
3,1180078562_object,1928-12-31,maleri,"Matisse, Henri",(1912),Olie på lærred,NaN,True,True,lærred,olie,#555555,#555555,#000000,#9B9B9B,#5A5A5A,NaN,NaN
4,1180044362_object,1928-12-31,maleri,"Matisse, Henri",1907-1908,Limfarve på lærred,NaN,True,True,lærred,limfarve,#9B9B9B,#5A5A5A,#5A5A5A,#9B9B9B,#555555,NaN,NaN


ok, so far so good. Det har været en kamp - men nu har jeg en 200, 18-dataframe

In [679]:
# Nu omdøber jeg lige et par kolonner for overblikkets skyld

df.rename(columns={'acquisition_date_precision':'acquisition'}, inplace=True)
df.rename(columns={'object_names_0_name':'type'}, inplace=True)
df.rename(columns={'production_0_creator':'creator'}, inplace=True)
df.rename(columns={'production_date_0_period':'production_period'}, inplace=True)
df.rename(columns={'techniques_0_technique':'technique'}, inplace=True)
df.rename(columns={'materials_0_material':'materials_0'}, inplace=True)
df.rename(columns={'materials_1_material':'materials_1'}, inplace=True)

In [680]:
df.head()

,id,acquisition,type,creator,production_period,technique,work_status_0,on_display,has_image,materials_0,materials_1,colors_0,colors_1,colors_2,colors_3,colors_4,techniques_1_technique,production_date_1_period
0,1180043169_object,1976-12-30,tegning,"Giersing, Harald",Ca. 1922,Blyant,kopi,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1180076925_object,1928-12-31,maleri,"Matisse, Henri",(1906),Olie på lærred,NaN,True,True,lærred,olie,#D2D2D2,#555555,#9B9B9B,#5A5A5A,#9B9B9B,NaN,NaN
2,1180064772_object,1936-12-31,maleri,"Matisse, Henri",(1905),Olie på lærred,NaN,True,True,lærred,olie,#863B7A,#F4C6A3,#555555,#9B9B9B,#037676,NaN,NaN
3,1180078562_object,1928-12-31,maleri,"Matisse, Henri",(1912),Olie på lærred,NaN,True,True,lærred,olie,#555555,#555555,#000000,#9B9B9B,#5A5A5A,NaN,NaN
4,1180044362_object,1928-12-31,maleri,"Matisse, Henri",1907-1908,Limfarve på lærred,NaN,True,True,lærred,limfarve,#9B9B9B,#5A5A5A,#5A5A5A,#9B9B9B,#555555,NaN,NaN


In [681]:
df.loc[df['creator'] == 'Matisse, Henri']

,id,acquisition,type,creator,production_period,technique,work_status_0,on_display,has_image,materials_0,materials_1,colors_0,colors_1,colors_2,colors_3,colors_4,techniques_1_technique,production_date_1_period
1,1180076925_object,1928-12-31,maleri,"Matisse, Henri",(1906),Olie på lærred,NaN,True,True,lærred,olie,#D2D2D2,#555555,#9B9B9B,#5A5A5A,#9B9B9B,NaN,NaN
2,1180064772_object,1936-12-31,maleri,"Matisse, Henri",(1905),Olie på lærred,NaN,True,True,lærred,olie,#863B7A,#F4C6A3,#555555,#9B9B9B,#037676,NaN,NaN
3,1180078562_object,1928-12-31,maleri,"Matisse, Henri",(1912),Olie på lærred,NaN,True,True,lærred,olie,#555555,#555555,#000000,#9B9B9B,#5A5A5A,NaN,NaN
4,1180044362_object,1928-12-31,maleri,"Matisse, Henri",1907-1908,Limfarve på lærred,NaN,True,True,lærred,limfarve,#9B9B9B,#5A5A5A,#5A5A5A,#9B9B9B,#555555,NaN,NaN
5,1180048692_object,1928-12-31,maleri,"Matisse, Henri",1906 - 1907,Olie på lærred,NaN,False,True,lærred,olie,#F4C6A3,#5A5A5A,#9B9B9B,#9B9B9B,#000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1180077680_object,1941-12-31,stregætsning,"Matisse, Henri",(1932),Stregætsning,del af serie,False,True,kinapapir,NaN,#FFFFFF,#FFFFFF,#FFFFFF,#FFFFFF,#D2D2D2,NaN,1932
196,1180077736_object,1941-12-31,stregætsning,"Matisse, Henri",(1932),Stregætsning,del af serie,False,True,japanpapir,NaN,#FFFFFF,#FFFFFF,#FFFFFF,#D2D2D2,#D2D2D2,NaN,1932
197,1180078107_object,1941-12-31,stregætsning,"Matisse, Henri",(1932),Stregætsning,del af serie,False,True,kinapapir,NaN,#FFFFFF,#FFFFFF,#FFFFFF,#FFFFFF,#D2D2D2,NaN,1932
198,1180082384_object,1941-12-31,stregætsning,"Matisse, Henri",(1932),Stregætsning,del af serie,False,True,kinapapir,NaN,#FFFFFF,#FFFFFF,#FFFFFF,#FFFFFF,#D2D2D2,NaN,1932


In [682]:
# Siden jeg kun er interesseret i at arbejde med værker af Matisse, vil jeg gerne af med de resultater, der (af uransagelige årsager), er dukket op med andre 'creators'
df.loc[df['creator'] != 'Matisse, Henri']
# og der var heldigvis kun to


,id,acquisition,type,creator,production_period,technique,work_status_0,on_display,has_image,materials_0,materials_1,colors_0,colors_1,colors_2,colors_3,colors_4,techniques_1_technique,production_date_1_period
0,1180043169_object,1976-12-30,tegning,"Giersing, Harald",Ca. 1922,Blyant,kopi,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1180078271_object,2004-03-26,video,"Just, Jesper",2004,"Video med lyd, 7:12 min.",del af serie,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [683]:
# Så dem tager vi ud af ligningen
df.drop([0,9], axis=0, inplace = True)

In [684]:
df.head(10)
# Og jeg sikrer mig, at de er ude (hvilket jeg kan gøre ved at se de første 10 resultater, siden vi fjernede række 0 og 9)

,id,acquisition,type,creator,production_period,technique,work_status_0,on_display,has_image,materials_0,materials_1,colors_0,colors_1,colors_2,colors_3,colors_4,techniques_1_technique,production_date_1_period
1,1180076925_object,1928-12-31,maleri,"Matisse, Henri",(1906),Olie på lærred,NaN,True,True,lærred,olie,#D2D2D2,#555555,#9B9B9B,#5A5A5A,#9B9B9B,NaN,NaN
2,1180064772_object,1936-12-31,maleri,"Matisse, Henri",(1905),Olie på lærred,NaN,True,True,lærred,olie,#863B7A,#F4C6A3,#555555,#9B9B9B,#037676,NaN,NaN
3,1180078562_object,1928-12-31,maleri,"Matisse, Henri",(1912),Olie på lærred,NaN,True,True,lærred,olie,#555555,#555555,#000000,#9B9B9B,#5A5A5A,NaN,NaN
4,1180044362_object,1928-12-31,maleri,"Matisse, Henri",1907-1908,Limfarve på lærred,NaN,True,True,lærred,limfarve,#9B9B9B,#5A5A5A,#5A5A5A,#9B9B9B,#555555,NaN,NaN
5,1180048692_object,1928-12-31,maleri,"Matisse, Henri",1906 - 1907,Olie på lærred,NaN,False,True,lærred,olie,#F4C6A3,#5A5A5A,#9B9B9B,#9B9B9B,#000000,NaN,NaN
6,1180010534_object,1928-12-31,maleri,"Matisse, Henri",(1905),Olie på lærred,studie,True,True,lærred,olie,#5A5A5A,#555555,#9B9B9B,#D2D2D2,#000000,NaN,NaN
7,1180061297_object,1928-12-31,maleri,"Matisse, Henri",(1904),Olie på lærred,NaN,False,True,lærred,olie,#5A5A5A,#9B9B9B,#9B9B9B,#D2D2D2,#000000,NaN,NaN
8,1180057569_object,1928-12-31,maleri,"Matisse, Henri",(1923),Olie på lærred,NaN,True,True,lærred,olie,#5A5A5A,#F4C6A3,#9B9B9B,#863B7A,#F4C6A3,NaN,NaN
10,1180011340_object,1928-12-31,maleri,"Matisse, Henri",(1906),Olie på træ,NaN,False,True,træ,olie,#F4C6A3,#FF9897,#D2D2D2,#555555,#9B9B9B,NaN,NaN
11,1180046872_object,1928-12-31,maleri,"Matisse, Henri",(1899),Olie på lærred,NaN,False,True,lærred,olie,#5A5A5A,#9B9B9B,#F4C6A3,#555555,#9B9B9B,NaN,NaN


In [688]:
df.dtypes

id                          object
acquisition                 object
type                        object
creator                     object
production_period           object
technique                   object
work_status_0               object
on_display                    bool
has_image                     bool
materials_0                 object
materials_1                 object
colors_0                    object
colors_1                    object
colors_2                    object
colors_3                    object
colors_4                    object
techniques_1_technique      object
production_date_1_period    object
dtype: object

In [690]:
df['id'].is_unique
# Og der er ingen dubleanter i sættet

True